In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Todo 1: EDA
1. Lưu ý khi data processing:
- Không có giá trị null
- Không có outliers

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
file_path = '/kaggle/input/ecommerce-customers/Ecommerce Customers.csv'
ecom_data = pd.read_csv(file_path, index_col = "Email")

# Todo 1: EDA

In [ ]:
ecom_data.head()

In [ ]:
ecom_data.info()

In [ ]:
ecom_data.describe()

In [ ]:
ecom_data.hist(grid=False, figsize=(10,10))

=> Conclusion for data processing: 
- No outlier
- No null value

In [ ]:
pd.plotting.scatter_matrix(ecom_data, alpha=0.5, figsize=(14,14))

It can be correlation between Yearly Amount Spent vs Time on App and Length of Membership

# Todo 2: Linear regressions

# Data Processing: y,X

In [ ]:
x = ecom_data.iloc[:,-5:-1]
y=ecom_data.iloc[:,-1]

In [ ]:
x.head()

# Split Train & Test Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

# Modeling

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train,y_train)

In [ ]:
# DataFrame.T is for Transpose
coef = pd.DataFrame([X_train.columns, reg.coef_]).T
coef = coef.rename(columns={0:'Attributes',1:'Coefficients'})
coef

# Evaluation

predictions y_pred=f(X_test)

In [ ]:
ytest_pred = reg.predict(X_test)

In [ ]:
from sklearn import metrics
lin_acc=metrics.r2_score(ytest_pred,y_test)
print("R^2: ",lin_acc)
print("MAE - Mean absolute error: ", metrics.mean_absolute_error(ytest_pred,y_test))
print("MSE - Mean squared error: ", metrics.mean_squared_error(ytest_pred,y_test))
print("RMSE: ",np.sqrt(metrics.mean_squared_error(ytest_pred,y_test)))
print("Max error: ",metrics.max_error(ytest_pred,y_test))

In [ ]:
x_ax = range(len(X_test))
plt.figure(figsize=(14,7))
plt.plot(x_ax, y_test, lw=1, color="Blue",label="Original")
plt.plot(x_ax, ytest_pred, lw=0.8, color="Red", label = "Predicted", marker="o", markersize=4)
plt.legend()
plt.show()

In [ ]:
df_plot = pd.DataFrame({'pred_spent':ytest_pred,'actual_spent':y_test})
df_plot.head()

In [ ]:
sns.jointplot(x='pred_spent', y='actual_spent',data=df_plot,kind='reg')

# Todo3: Clustering with K-Means

In [ ]:
from sklearn.cluster import KMeans
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

In [ ]:
Xclus = ecom_data.iloc[:,-5:-1]
Xclus.head()

In [ ]:
cs = []
for i in range(1, 7):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(Xclus)
    cs.append(kmeans.inertia_)
plt.plot(range(1, 7), cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

In [ ]:
# create cluster feature
kmeans = KMeans(n_clusters=3)
Xclus['Cluster']=kmeans.fit_predict(Xclus)
Xclus['Cluster']=Xclus['Cluster'].astype("category")
Xclus.head()

In [ ]:
sns.relplot(x='Length of Membership', y='Time on App',hue="Cluster",data=Xclus)

In [ ]:
sns.relplot(x='Avg. Session Length', y='Time on Website',hue="Cluster",data=Xclus)

In [ ]:
Xclus["Yearly Amount Spent"] = ecom_data["Yearly Amount Spent"]
sns.catplot(x="Yearly Amount Spent", y="Cluster", data=Xclus, kind="boxen", height=6);

# Conclution
We should focus on cluster 1, with the characterics as:


In [ ]:
Xclus1 = Xclus[Xclus['Cluster'] == 1]
Xclus1.describe()